In [15]:
from pdbfixer import PDBFixer
from openmm.app import *
from openmm import *
from openmm.unit import *
from sys import stdout
import numpy as np
import pandas as pd
import re
import time
from datetime import datetime
from xml.etree.ElementTree import Element
from xml.etree.ElementTree import SubElement
from xml.etree.ElementTree import ElementTree
import math

In [16]:
# 输入文件
profile="protein.pdb"
liggro="LIG.gro"
ligtop="LIG_GMX.top"
ligxml="lig.xml"
ph=7.0

# 系统设置
nonbondedMethod = PME
nonbondedCutoff = 1.0*nanometers
constraints = HBonds
rigidWater = True
hydrogenMass = 1.5*amu
ewaldErrorTolerance = 0.0005
constraintTolerance = 0.000001

# 积分选项
dt = 0.002*picoseconds
temperature = 306*kelvin
friction = 1.0/picosecond
pressure = 1.0*atmospheres
barostatInterval = 100

# 模拟选项
steps = 100000
equilibrationSteps = 1000
platform = Platform.getPlatformByName('CUDA')
recordinpdb=False
recordindcd=True
stdreport=True
pdbreporter = PDBReporter("trajectory.pdb",5000)
dcdReporter = DCDReporter('trajectory.dcd', 5000)
stdReporter = StateDataReporter(stdout, 1000, totalSteps=steps,
    step=True, speed=True, progress=True,kineticEnergy=True, potentialEnergy=True, temperature=True, separator='\t')
dataReporter = StateDataReporter("log.txt", 1000, totalSteps=steps,
    step=True, speed=True, progress=True,kineticEnergy=True, potentialEnergy=True, temperature=True, separator='\t')
checkpointReporter = CheckpointReporter('checkpoint.chk', 50000)



In [17]:
# 处理蛋白
print("处理蛋白中--------------------------------")
fixer=PDBFixer(filename=profile)
fixer.findMissingResidues()
fixer.findNonstandardResidues()
fixer.replaceNonstandardResidues()
fixer.removeHeterogens(True)
fixer.findMissingAtoms()
fixer.addMissingAtoms()
fixer.addMissingHydrogens(ph)
PDBFile.writeFile(fixer.topology,fixer.positions,open("protein_fixed.pdb","w"))
print("蛋白处理完成------------------------------")

处理蛋白中------------


In [18]:
# 处理配体
print("处理配体中--------------------------------")
gro=GromacsGroFile(liggro)
top=GromacsTopFile(ligtop)
system = top.createSystem(nonbondedMethod=NoCutoff)
integrator = LangevinMiddleIntegrator(300*kelvin, 1/picosecond, 0.004*picoseconds)
simulation = Simulation(top.topology, system, integrator)
simulation.context.setPositions(gro.positions)
simulation.minimizeEnergy()
positions=simulation.context.getState(getPositions=True).getPositions()
PDBFile.writeFile(simulation.topology,positions,open("lig.pdb","w"))

In [19]:
# 合并蛋白和配体并生成盒子
print("合并蛋白-配体并生成模拟体系---------------")
lig=PDBFile("lig.pdb")
pro=PDBFile("protein_fixed.pdb")
modeller=Modeller(pro.topology,pro.positions)
modeller.add(lig.topology,lig.positions)
forcefield=ForceField("amber14-all.xml","amber14/tip3p.xml",ligxml)
modeller.addSolvent(padding=8*angstrom,forcefield=forcefield,neutralize=True)
mergedTopology=modeller.topology
mergedPositions=modeller.positions
PDBFile.writeFile(mergedTopology,mergedPositions,open("complex.pdb","w"))


合并蛋白-配体并生成模拟体系---------


In [20]:
# 构建模拟系统
print("模拟准备中--------------------------------")
system=forcefield.createSystem(mergedTopology,nonbondedMethod=nonbondedMethod,nonbondedCutoff=nonbondedCutoff,
        constraints=constraints,rigidWater=rigidWater, ewaldErrorTolerance=ewaldErrorTolerance,hydrogenMass=hydrogenMass)
system.addForce(MonteCarloBarostat(pressure, temperature, barostatInterval))
integrator = LangevinMiddleIntegrator(temperature, friction, dt)
integrator.setConstraintTolerance(constraintTolerance)
simulation = Simulation(mergedTopology, system, integrator, platform)
simulation.context.setPositions(mergedPositions)

模拟准备中---------------


In [21]:
# 最小化
print("能量最小化--------------------------------")
simulation.minimizeEnergy()


能量最小化------------


In [22]:
# 预平衡
print("平衡中------------------------------------")
simulation.context.setVelocitiesToTemperature(temperature)
simulation.step(equilibrationSteps)

平衡中---------


In [23]:
# 成品模拟
print("Simulating--------------------------------")
if recordinpdb:
    simulation.reporters.append(pdbreporter)
if recordindcd:
    simulation.reporters.append(dcdReporter)
if stdreport:
    simulation.reporters.append(stdReporter)
simulation.reporters.append(dataReporter)
simulation.reporters.append(checkpointReporter)
simulation.currentStep = 0
simulation.step(steps)

Simulating-----------
#"Progress (%)"	"Step"	"Potential Energy (kJ/mole)"	"Kinetic Energy (kJ/mole)"	"Temperature (K)"	"Speed (ns/day)"
1.0%	1000	-1475235.0929329796	295248.14293694176	297.9030245092713	0
2.0%	2000	-1462387.306404342	302507.9292460726	305.228091069446	58
3.0%	3000	-1460528.3182784254	303362.5352094023	306.0903816793279	65.7
4.0%	4000	-1457732.3182784254	302322.2686517459	305.0407610085964	68.4
5.0%	5000	-1461246.2996952506	303435.1211558023	306.163620321083	65.1
6.0%	6000	-1461067.9328578245	302196.5645154257	304.9139265363094	66.8
7.0%	7000	-1460825.1439040322	301626.88402334857	304.3391234574296	68.1
8.0%	8000	-1461209.9049189473	302556.4255117976	305.2770234152795	69
9.0%	9000	-1462883.4241621615	303378.56306808913	306.1065536610776	69.6
10.0%	10000	-1463489.9241621615	303372.29675512714	306.1002310012114	66.1
11.0%	11000	-1464424.2554459833	304627.4822000516	307.36670311732183	66.8
12.0%	12000	-1463061.2554459833	303617.99502138066	306.3481385948224	67.4
13.0%	1300